# Sample 05: బహుళ-ఏజెంట్ సమన్వయ వ్యవస్థ

ఈ నోట్‌బుక్ Microsoft Foundry Local ఉపయోగించి AI-శక్తివంతమైన ఏజెంట్ వ్యవస్థలను నిర్మించడానికి ఒక సున్నితమైన బహుళ-ఏజెంట్ నిర్మాణాన్ని ప్రదర్శిస్తుంది.

## అవలోకనం

ఈ నమూనా ప్రత్యేక ఏజెంట్లను సమన్వయపరచే **బహుళ-ఏజెంట్ సమన్వయకర్త**ను అమలు చేస్తుంది:

- 🔍 **రెట్రీవల్ ఏజెంట్**: జ్ఞాన మూలాల నుండి సంబంధిత సమాచారాన్ని తీసుకుంటుంది
- 🧠 **రీజనింగ్ ఏజెంట్**: దశలవారీ విశ్లేషణ మరియు తార్కిక కారణాన్ని నిర్వహిస్తుంది
- ⚡ **ఎగ్జిక్యూషన్ ఏజెంట్**: నిర్మిత ఫార్మాట్లలో కార్యాచరణ ప్రణాళికలను సృష్టిస్తుంది
- 🎯 **సమన్వయకర్త**: మొత్తం ఏజెంట్ వర్క్‌ఫ్లోను సమన్వయపరుస్తుంది


## ఆర్కిటెక్చర్ నమూనా

```
User Goal → Coordinator
     ↓
1. Retrieval Agent → Context
     ↓
2. Reasoning Agent → Decision
     ↓
3. Execution Agent → Actions
     ↓
Structured Result
```


## ముందస్తు అవసరాలు మరియు సెటప్

మీకు సామర్థ్యమున్న మోడల్‌తో Foundry Local నడుస్తున్నదని నిర్ధారించుకోండి:


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## లైబ్రరీలు మరియు కాన్ఫిగరేషన్‌ను దిగుమతి చేసుకోండి


In [ ]:
import os
import json
import time
from typing import Dict, Any, List
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

# Configuration
MODEL_ALIAS = "phi-4-mini"  # Change to your preferred model
BASE_URL = "http://localhost:8000"
API_KEY = ""

## Foundry క్లయింట్ సెటప్

అన్ని ఏజెంట్ల కోసం ఒక పంచుకున్న క్లయింట్ సృష్టించండి:


In [ ]:
class FoundryClient:
    """Shared client for all specialist agents."""
    
    def __init__(self, model_alias: str = MODEL_ALIAS):
        self.client = None
        self.model_name = None
        self.model_alias = model_alias
        self._initialize_client()
    
    def _initialize_client(self):
        """Initialize OpenAI client with Foundry Local or fallback configuration."""
        if FOUNDRY_SDK_AVAILABLE:
            try:
                print(f"🔄 Initializing Foundry Local with model: {self.model_alias}...")
                manager = FoundryLocalManager(self.model_alias)
                model_info = manager.get_model_info(self.model_alias)
                
                self.client = OpenAI(
                    base_url=manager.endpoint,
                    api_key=manager.api_key
                )
                self.model_name = model_info.id
                print(f"✅ Foundry Local SDK initialized with model: {self.model_name}")
                return
            except Exception as e:
                print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
        
        # Fallback to manual configuration
        self.client = OpenAI(
            base_url=f"{BASE_URL}/v1",
            api_key=API_KEY
        )
        self.model_name = self.model_alias
        print(f"🔧 Manual configuration initialized with model: {self.model_name}")
    
    def chat(self, messages: List[Dict[str, str]], max_tokens: int = 300, temperature: float = 0.4) -> str:
        """Send chat completion request to the model."""
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating response: {str(e)}"
    
    def check_health(self) -> bool:
        """Check if the client is working properly."""
        try:
            test_response = self.chat(
                [{"role": "user", "content": "Say 'OK'"}],
                max_tokens=5
            )
            return "OK" in test_response and "Error" not in test_response
        except:
            return False

# Initialize the shared client
print("Initializing Foundry Client...")
foundry_client = FoundryClient()

# Health check
if foundry_client.check_health():
    print("✅ Client health check passed!")
else:
    print("❌ Client health check failed. Please ensure Foundry Local is running with a model.")

## ప్రత్యేక ఏజెంట్ తరగతులు

ప్రతి ఏజెంట్ నిర్దిష్ట జ్ఞానాత్మక పనుల కోసం ఆప్టిమైజ్ చేయబడింది:


In [ ]:
class RetrievalAgent:
    """Agent specialized in retrieving relevant information from knowledge sources."""
    
    SYSTEM = """You are a specialized retrieval agent. Your job is to extract and retrieve 
    the most relevant information from knowledge sources based on a given query. Focus on key facts, 
    data points, and contextual information that would be useful for decision-making."""
    
    def __init__(self, client: FoundryClient):
        self.client = client
    
    def run(self, query: str) -> str:
        """Retrieve relevant information based on the query."""
        messages = [
            {"role": "system", "content": self.SYSTEM},
            {
                "role": "user", 
                "content": f"""Query: {query}

Retrieve the most relevant key facts, data points, and contextual information that would 
help answer this query or support decision-making around it. Provide specific, actionable 
information rather than general statements."""
            }
        ]
        return self.client.chat(messages)


class ReasoningAgent:
    """Agent specialized in step-by-step analysis and reasoning."""
    
    SYSTEM = """You are a specialized reasoning agent. Your job is to analyze inputs 
    step-by-step and produce structured, logical conclusions. Break down complex problems 
    into manageable parts and provide clear reasoning for your conclusions."""
    
    def __init__(self, client: FoundryClient):
        self.client = client
    
    def run(self, context: str, question: str) -> str:
        """Analyze context and question to produce structured conclusions."""
        messages = [
            {"role": "system", "content": self.SYSTEM},
            {
                "role": "user", 
                "content": f"""Context:
{context}

Question: {question}

Analyze this step-by-step and provide a structured, logical conclusion with clear reasoning. 
Break down the problem, consider different angles, and provide a well-reasoned decision or recommendation."""
            }
        ]
        return self.client.chat(messages, max_tokens=400)


class ExecutionAgent:
    """Agent specialized in creating actionable execution plans."""
    
    SYSTEM = """You are a specialized execution agent. Your job is to transform decisions 
    and conclusions into concrete, actionable steps. Always format your response as valid JSON 
    with an array of action items. Each action should be specific, measurable, and achievable."""
    
    def __init__(self, client: FoundryClient):
        self.client = client
    
    def run(self, decision: str) -> str:
        """Transform decision into actionable steps in JSON format."""
        messages = [
            {"role": "system", "content": self.SYSTEM},
            {
                "role": "user", 
                "content": f"""Decision/Conclusion:
{decision}

Create 3-5 specific, actionable steps to implement this decision. Format as JSON with this structure:
{{
  "actions": [
    {{
      "step": 1,
      "description": "Specific action description",
      "priority": "high/medium/low",
      "timeline": "timeframe for completion",
      "resources": ["required resources or people"]
    }}
  ]
}}"""
            }
        ]
        return self.client.chat(messages, max_tokens=400, temperature=0.3)

print("✅ Agent classes defined")

## బహుళ-ఏజెంట్ సమన్వయకర్త

సమన్వయకర్త అన్ని ఏజెంట్లను సమన్వయించి సంక్లిష్ట పనులను నిర్వహిస్తుంది:


In [ ]:
class Coordinator:
    """Multi-agent coordinator that orchestrates specialist agents to handle complex tasks."""
    
    def __init__(self, client: FoundryClient):
        """Initialize the coordinator with specialist agents."""
        self.client = client
        self.retrieval = RetrievalAgent(client)
        self.reasoning = ReasoningAgent(client)
        self.execution = ExecutionAgent(client)
    
    def handle(self, user_goal: str) -> Dict[str, Any]:
        """
        Orchestrate multiple agents to handle a complex user goal.
        
        Args:
            user_goal: The user's high-level goal or request
            
        Returns:
            Dictionary containing the goal, context, decision, and actions
        """
        print(f"🎯 **Coordinator:** Processing goal: {user_goal}")
        print("=" * 60)
        
        start_time = time.time()
        
        # Step 1: Retrieve relevant context
        print("📚 **Step 1:** Retrieving context...")
        context = self.retrieval.run(user_goal)
        print(f"   ✅ Context retrieved ({len(context)} chars)")
        print(f"   📄 Preview: {context[:150]}...\n")
        
        # Step 2: Analyze and reason about the context
        print("🧠 **Step 2:** Analyzing and reasoning...")
        decision = self.reasoning.run(context, user_goal)
        print(f"   ✅ Analysis completed ({len(decision)} chars)")
        print(f"   💡 Preview: {decision[:150]}...\n")
        
        # Step 3: Create actionable execution plan
        print("⚡ **Step 3:** Creating execution plan...")
        actions = self.execution.run(decision)
        print(f"   ✅ Execution plan created ({len(actions)} chars)")
        
        # Try to parse actions as JSON for preview
        try:
            actions_json = json.loads(actions)
            action_count = len(actions_json.get('actions', []))
            print(f"   📋 Actions planned: {action_count}\n")
        except:
            print(f"   📋 Actions: {actions[:100]}...\n")
        
        end_time = time.time()
        processing_time = end_time - start_time
        
        result = {
            "goal": user_goal,
            "context": context,
            "decision": decision,
            "actions": actions,
            "agent_flow": ["retrieval", "reasoning", "execution"],
            "processing_time": processing_time,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        print(f"✅ **Coordination Complete** (⏱️ {processing_time:.2f}s)")
        return result
    
    def handle_with_feedback(self, user_goal: str, feedback_rounds: int = 1) -> Dict[str, Any]:
        """
        Handle a goal with multiple feedback rounds for refinement.
        
        Args:
            user_goal: The user's high-level goal or request
            feedback_rounds: Number of feedback rounds to perform
            
        Returns:
            Dictionary containing the refined result
        """
        result = self.handle(user_goal)
        
        for round_num in range(feedback_rounds):
            print(f"\n🔄 **Feedback Round {round_num + 1}:**")
            print("-" * 40)
            
            # Use reasoning agent to refine the execution plan
            refinement_prompt = f"""
            Original Goal: {user_goal}
            Current Decision: {result['decision']}
            Current Actions: {result['actions']}
            
            Review the above and suggest improvements or refinements to make the execution plan more effective.
            Consider potential challenges, resource optimization, and success metrics.
            """
            
            refined_decision = self.reasoning.run(result['context'], refinement_prompt)
            refined_actions = self.execution.run(refined_decision)
            
            result['decision'] = refined_decision
            result['actions'] = refined_actions
            result['refinement_rounds'] = round_num + 1
            
            print(f"   ✅ Round {round_num + 1} refinement completed")
        
        return result

# Initialize coordinator
coordinator = Coordinator(foundry_client)
print("✅ Multi-agent coordinator initialized")

## Example 1: వ్యాపార ప్రణాళిక

వ్యాపార ప్రణాళిక లక్ష్యంతో కోఆర్డినేటర్‌ను పరీక్షిద్దాం:


In [ ]:
# Business planning example
business_goal = "Create a plan to onboard 5 new customers this month"

print(f"🚀 **Business Planning Example**")
print(f"📋 Goal: {business_goal}")
print("=" * 80)

business_result = coordinator.handle(business_goal)

print("\n📊 **Final Result Summary:**")
print("=" * 50)
print(f"🎯 **Goal:** {business_result['goal']}")
print(f"⏱️ **Processing Time:** {business_result['processing_time']:.2f} seconds")
print(f"🕒 **Timestamp:** {business_result['timestamp']}")

print(f"\n📚 **Context (Retrieval Agent):**")
print(business_result['context'])

print(f"\n🧠 **Decision (Reasoning Agent):**")
print(business_result['decision'])

print(f"\n⚡ **Actions (Execution Agent):**")
print(business_result['actions'])

## Example 2: వ్యూహ అభివృద్ధి

మరింత సంక్లిష్టమైన వ్యూహ అభివృద్ధి లక్ష్యంతో పరీక్షించండి:


In [ ]:
# Strategy development example
strategy_goal = "Develop a strategy to improve team productivity by 20% while maintaining work-life balance"

print(f"🎯 **Strategy Development Example**")
print(f"📋 Goal: {strategy_goal}")
print("=" * 80)

strategy_result = coordinator.handle(strategy_goal)

print("\n📊 **Structured Action Plan:**")
print("=" * 40)

# Try to parse and display actions in a structured format
try:
    actions_data = json.loads(strategy_result['actions'])
    if 'actions' in actions_data:
        for i, action in enumerate(actions_data['actions'], 1):
            print(f"\n📌 **Action {i}:**")
            print(f"   📝 Description: {action.get('description', 'N/A')}")
            print(f"   🔥 Priority: {action.get('priority', 'N/A')}")
            print(f"   ⏰ Timeline: {action.get('timeline', 'N/A')}")
            print(f"   🛠️ Resources: {', '.join(action.get('resources', ['N/A']))}")
    else:
        print(strategy_result['actions'])
except json.JSONDecodeError:
    print("Raw actions output:")
    print(strategy_result['actions'])

## Example 3: ఫీడ్‌బ్యాక్ లూప్ మెరుగుదల

పునరావృత మెరుగుదల కోసం ఫీడ్‌బ్యాక్ యంత్రాంగాన్ని ప్రదర్శించండి:


In [9]:
# Feedback loop example
feedback_goal = "Design a customer feedback collection system for a software product"

print(f"🔄 **Feedback Loop Refinement Example**")
print(f"📋 Goal: {feedback_goal}")
print("=" * 80)

# Process with 2 feedback rounds
feedback_result = coordinator.handle_with_feedback(feedback_goal, feedback_rounds=2)

print("\n🏆 **Final Refined Result:**")
print("=" * 50)
print(f"🎯 **Goal:** {feedback_result['goal']}")
print(f"🔄 **Refinement Rounds:** {feedback_result.get('refinement_rounds', 0)}")
print(f"⏱️ **Total Processing Time:** {feedback_result['processing_time']:.2f} seconds")

print(f"\n🧠 **Final Decision:**")
print(feedback_result['decision'])

print(f"\n⚡ **Final Action Plan:**")
print(feedback_result['actions'])

   ✅ Round 2 refinement completed

🏆 **Final Refined Result:**
🎯 **Goal:** Design a customer feedback collection system for a software product
🔄 **Refinement Rounds:** 2
⏱️ **Total Processing Time:** 559.18 seconds

🧠 **Final Decision:**
The execution plan for designing a customer feedback collection system for a software product is comprehensive, but there are areas where it could be refined for better effectiveness. Here are some suggestions:

1. **Review of Existing Feedback Mechanisms**: This step is crucial as it sets the direction for the feedback collection system. However, it could be more effective if it also includes a review of existing feedback mechanisms and their shortcomings. This will help in understanding what can be improved.

2. **Survey or Focus Group for Feedback Channels**: While the plan includes a variety of feedback channels, it could be beneficial to conduct a survey or a small focus group with a sample of the target audience to understand their preferred feed

## ఇంటరాక్టివ్ ఏజెంట్ పరీక్ష

వారి ప్రత్యేక సామర్థ్యాలను అర్థం చేసుకోవడానికి వ్యక్తిగత ఏజెంట్లను వేరుగా పరీక్షించండి:


In [10]:
def test_individual_agents(query: str):
    """Test each agent individually with the same query."""
    print(f"🧪 **Individual Agent Testing**")
    print(f"❓ Query: {query}")
    print("=" * 60)
    
    # Test Retrieval Agent
    print("\n🔍 **Retrieval Agent:**")
    retrieval_result = coordinator.retrieval.run(query)
    print(retrieval_result)
    
    # Test Reasoning Agent (using retrieval result as context)
    print("\n🧠 **Reasoning Agent:**")
    reasoning_result = coordinator.reasoning.run(retrieval_result, query)
    print(reasoning_result)
    
    # Test Execution Agent (using reasoning result)
    print("\n⚡ **Execution Agent:**")
    execution_result = coordinator.execution.run(reasoning_result)
    print(execution_result)

# Test with a simple query
test_query = "How can we reduce customer support response time?"
test_individual_agents(test_query)

🧪 **Individual Agent Testing**
❓ Query: How can we reduce customer support response time?

🔍 **Retrieval Agent:**
1. Implementing AI-powered chatbots: AI-powered chatbots can handle common customer queries, reducing the workload on human agents and speeding up response times. According to a study by Accenture, AI chatbots can handle 80% of customer interactions, freeing up human agents to handle more complex issues.

2. Streamlining the support process: Simplifying the support process and removing unnecessary steps can help reduce response times. This could involve consolidating support channels, creating self-help resources, or automating certain processes.

3. Increasing support staff: Hiring additional support staff or training existing staff to handle more complex issues can help reduce response times. A study by Forrester found that increasing the number of support agents by just 10% can reduce average response time by 20%.

4. Prioritizing urgent issues: Prioritizing urgent issue

## కస్టమ్ గోల్ టెస్టింగ్

మీ స్వంత లక్ష్యాలను పరీక్షించడానికి ఈ సెల్‌ను ఉపయోగించండి:


In [ ]:
# Custom goal testing - modify the goal below
custom_goal = "Create a training program for new AI engineers joining our company"

print(f"🎨 **Custom Goal Testing**")
print(f"📋 Your Goal: {custom_goal}")
print("=" * 60)

# Choose processing method
use_feedback = True  # Set to True for feedback rounds, False for basic processing
feedback_rounds = 1  # Number of feedback rounds if use_feedback is True

if use_feedback:
    custom_result = coordinator.handle_with_feedback(custom_goal, feedback_rounds=feedback_rounds)
    print(f"\n✨ **Result with {feedback_rounds} feedback round(s):**")
else:
    custom_result = coordinator.handle(custom_goal)
    print(f"\n✨ **Basic Result:**")

print("=" * 50)
print(f"📚 **Context:** {custom_result['context'][:200]}...")
print(f"\n🧠 **Decision:** {custom_result['decision'][:200]}...")
print(f"\n⚡ **Actions:** {custom_result['actions'][:200]}...")

# Show processing stats
print(f"\n📊 **Statistics:**")
print(f"   ⏱️ Processing Time: {custom_result['processing_time']:.2f}s")
print(f"   🔄 Refinement Rounds: {custom_result.get('refinement_rounds', 0)}")
print(f"   📏 Total Content Length: {len(custom_result['context']) + len(custom_result['decision']) + len(custom_result['actions'])} chars")

🎨 **Custom Goal Testing**
📋 Your Goal: Create a training program for new AI engineers joining our company
🎯 **Coordinator:** Processing goal: Create a training program for new AI engineers joining our company
📚 **Step 1:** Retrieving context...
   ✅ Context retrieved (1408 chars)
   📄 Preview: 1. **Program Structure**: A modular program with a mix of theoretical and practical sessions. Modules could include:
   - Introduction to AI and Machi...

🧠 **Step 2:** Analyzing and reasoning...


## పనితీరు విశ్లేషణ

బహుళ-ఏజెంట్ సిస్టమ్ యొక్క పనితీరును విశ్లేషించండి:


In [ ]:
def performance_benchmark(goals: List[str], iterations: int = 2) -> Dict[str, Any]:
    """Benchmark the coordinator performance with multiple goals."""
    results = []
    
    print(f"📊 **Performance Benchmark**")
    print(f"🎯 Goals: {len(goals)}")
    print(f"🔄 Iterations per goal: {iterations}")
    print("=" * 50)
    
    for i, goal in enumerate(goals, 1):
        print(f"\n🎯 **Goal {i}:** {goal[:50]}...")
        goal_times = []
        
        for j in range(iterations):
            print(f"   🔄 Iteration {j+1}/{iterations}...", end=" ")
            start_time = time.time()
            
            try:
                result = coordinator.handle(goal)
                end_time = time.time()
                processing_time = end_time - start_time
                goal_times.append(processing_time)
                print(f"✅ {processing_time:.2f}s")
            except Exception as e:
                print(f"❌ Error: {e}")
        
        if goal_times:
            avg_time = sum(goal_times) / len(goal_times)
            results.append({
                "goal": goal,
                "avg_time": avg_time,
                "min_time": min(goal_times),
                "max_time": max(goal_times),
                "times": goal_times
            })
    
    return results

# Benchmark with different types of goals
benchmark_goals = [
    "Create a social media marketing strategy",
    "Improve employee onboarding process",
    "Design a mobile app user interface",
    "Plan a product launch campaign"
]

benchmark_results = performance_benchmark(benchmark_goals, iterations=2)

# Display benchmark summary
print("\n🏆 **Benchmark Summary:**")
print("=" * 50)
for result in benchmark_results:
    print(f"📝 {result['goal'][:40]}...")
    print(f"   ⏱️ Average: {result['avg_time']:.2f}s")
    print(f"   ⚡ Fastest: {result['min_time']:.2f}s")
    print(f"   🐌 Slowest: {result['max_time']:.2f}s")
    print()

if benchmark_results:
    overall_avg = sum(r['avg_time'] for r in benchmark_results) / len(benchmark_results)
    print(f"📊 **Overall Average Processing Time:** {overall_avg:.2f}s")

## ఉత్పత్తి అమరిక సహాయకుడు

ఉత్పత్తి ఉపయోగానికి కోఆర్డినేటర్‌ను ఎలా చుట్టుకోవాలో ఉదాహరణ:


In [ ]:
class ProductionCoordinator:
    """Production-ready wrapper for the multi-agent coordinator."""
    
    def __init__(self, model_alias: str = "phi-4-mini"):
        self.client = FoundryClient(model_alias)
        self.coordinator = Coordinator(self.client)
        self.request_count = 0
        self.total_processing_time = 0
    
    def process_goal(self, goal: str, include_feedback: bool = False, feedback_rounds: int = 1) -> Dict[str, Any]:
        """Process a goal with production monitoring."""
        self.request_count += 1
        
        try:
            if include_feedback:
                result = self.coordinator.handle_with_feedback(goal, feedback_rounds=feedback_rounds)
            else:
                result = self.coordinator.handle(goal)
            
            self.total_processing_time += result['processing_time']
            
            # Add production metadata
            result['request_id'] = self.request_count
            result['status'] = 'success'
            result['model'] = self.client.model_name
            
            return result
            
        except Exception as e:
            return {
                'request_id': self.request_count,
                'status': 'error',
                'error': str(e),
                'goal': goal,
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
            }
    
    def get_stats(self) -> Dict[str, Any]:
        """Get production statistics."""
        avg_processing_time = self.total_processing_time / max(1, self.request_count)
        
        return {
            'total_requests': self.request_count,
            'total_processing_time': self.total_processing_time,
            'average_processing_time': avg_processing_time,
            'model': self.client.model_name,
            'client_healthy': self.client.check_health()
        }

# Example production usage
prod_coordinator = ProductionCoordinator()

# Process a goal
prod_goal = "Create a quarterly business review presentation"
prod_result = prod_coordinator.process_goal(prod_goal)

print(f"🏭 **Production Processing Result:**")
print(f"📊 Status: {prod_result['status']}")
print(f"🔢 Request ID: {prod_result['request_id']}")
print(f"⏱️ Processing Time: {prod_result.get('processing_time', 'N/A')}s")
print(f"🤖 Model: {prod_result.get('model', 'N/A')}")

# Show production stats
stats = prod_coordinator.get_stats()
print(f"\n📊 **Production Statistics:**")
print(f"   📈 Total Requests: {stats['total_requests']}")
print(f"   ⏱️ Average Processing Time: {stats['average_processing_time']:.2f}s")
print(f"   💚 Client Health: {'✅ Healthy' if stats['client_healthy'] else '❌ Unhealthy'}")

## సారాంశం మరియు ఉత్తమ ఆచారాలు

ఈ నోట్‌బుక్ ఒక సున్నితమైన బహుళ-ఏజెంట్ ఆర్కెస్ట్రేషన్ సిస్టమ్‌ను ప్రదర్శించింది:

### ✅ ప్రదర్శించిన ముఖ్య లక్షణాలు

1. **🏗️ ఏజెంట్ ప్రత్యేకత**: ప్రతి ఏజెంట్ నిర్దిష్ట జ్ఞానాత్మక పనుల కోసం ఆప్టిమైజ్ చేయబడింది  
2. **🎯 వర్క్‌ఫ్లో ఆర్కెస్ట్రేషన్**: బహుళ-దశ ప్రాసెసింగ్ సమన్వయం  
3. **📋 నిర్మిత అవుట్‌పుట్**: JSON-ఆకృతిలో చర్యా ప్రణాళికలు  
4. **🔄 ఫీడ్‌బ్యాక్ లూప్స్**: బహుళ-దశ సవరణ సామర్థ్యాలు  
5. **⚡ పనితీరు పర్యవేక్షణ**: ప్రాసెసింగ్ సమయం మరియు ఆరోగ్య తనిఖీలు  
6. **🏭 ఉత్పత్తి సిద్ధం**: మానిటరింగ్‌తో ఎంటర్ప్రైజ్-గ్రేడ్ రాపర్  

### 🧠 ఏజెంట్ పాత్రల సారాంశం

| ఏజెంట్ | ఉద్దేశ్యం | ఇన్‌పుట్ | అవుట్‌పుట్ |
|-------|---------|-------|--------|
| **🔍 రిట్రీవల్** | సంబంధిత సమాచారం తీసుకోవడం | వినియోగదారు ప్రశ్న | సందర్భానుగుణమైన వాస్తవాలు మరియు డేటా |
| **🧠 రీజనింగ్** | తార్కిక విశ్లేషణ | సందర్భం + ప్రశ్న | నిర్మిత నిర్ణయం |
| **⚡ ఎగ్జిక్యూషన్** | చర్యా ప్రణాళికలు సృష్టించడం | నిర్ణయం | JSON చర్యా దశలు |
| **🎯 కోఆర్డినేటర్** | వర్క్‌ఫ్లోను ఆర్కెస్ట్రేట్ చేయడం | వినియోగదారు లక్ష్యం | పూర్తి ఫలితం |

### 🚀 ఉపయోగాల సందర్భాలు

- **వ్యాపార ప్రణాళిక**: వ్యూహాత్మక ప్రణాళిక మరియు అమలు  
- **ప్రాజెక్ట్ నిర్వహణ**: పనుల విభజన మరియు షెడ్యూలింగ్  
- **సంశోధన**: సమాచారం సేకరణ మరియు విశ్లేషణ  
- **నిర్ణయ మద్దతు**: సంక్లిష్ట నిర్ణయ ప్రక్రియలు  
- **వర్క్‌ఫ్లో ఆటోమేషన్**: బహుళ-దశ వ్యాపార ప్రక్రియలు  

### 💡 ఉత్తమ ఆచారాలు

1. **🎯 ఏజెంట్ ఒక్కోటి స్పష్టమైన బాధ్యత కలిగి ఉండాలి**  
2. **🔗 స్పష్టమైన ఇంటర్‌ఫేస్‌లు**: ప్రమాణీకృత ఇన్‌పుట్/అవుట్‌పుట్ ఫార్మాట్లు  
3. **🛡️ లోపాల నిర్వహణ**: వైఫల్యాలపై సున్నితమైన తగ్గింపు  
4. **📊 పర్యవేక్షణ**: సమగ్ర లాగింగ్ మరియు పనితీరు ట్రాకింగ్  
5. **🔄 ఫీడ్‌బ్యాక్ లూప్స్**: పునరావృత మెరుగుదల యంత్రాంగాలు  
6. **⚖️ లోడ్ బ్యాలెన్సింగ్**: స్వతంత్ర పనుల కోసం సమాంతర ప్రాసెసింగ్ పరిగణన  

### 🔮 తదుపరి దశలు

- **🔧 ఫంక్షన్ కాలింగ్**: బాహ్య APIలు మరియు టూల్స్‌తో సమగ్రత  
- **🧠 మెమరీ సిస్టమ్స్**: ఏజెంట్ల కోసం స్థిరమైన మెమరీ జోడింపు  
- **🎭 ప్రత్యేక మోడల్స్**: వేర్వేరు ఏజెంట్ల కోసం వేర్వేరు మోడల్స్ ఉపయోగించడం  
- **👥 హ్యూమన్-ఇన్-ది-లూప్**: మానవ సమీక్ష మరియు ఆమోద దశలను జోడించడం  
- **📊 అధునాతన విశ్లేషణలు**: సమగ్ర పర్యవేక్షణ మరియు మెట్రిక్స్  

ఈ బహుళ-ఏజెంట్ సిస్టమ్ ప్రత్యేక ఏజెంట్ల బలాలను కలిపి, Microsoft Foundry Localతో స్థానిక ఇన్ఫరెన్స్ యొక్క గోప్యత మరియు పనితీరు లాభాలను నిలుపుకుంటూ సున్నితమైన AI వర్క్‌ఫ్లోలను ఎలా నిర్మించాలో ప్రదర్శిస్తుంది.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారుల బాధ్యత మేము తీసుకోము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
